In [199]:
#Multinomial Naive Bayes
##########################################################################
##Learning Process
##Preprocess the Data
##Generate negative review
##open mega negative doc
f = open("NegVocab.txt")
lines = readlines(f)
close(f)
#chomp each line
for i = 1:length(lines)
    lines[i] = chomp(lines[i])
end
#create negative dictionary with tf
NegDictionary = Dict()
for i = 1:length(lines)
    tempArray = split(lines[i])
    tempWord = ""
    tempWord = tempArray[1]
    #tf
    tempTF = parse(tempArray[2])
    NegDictionary[tempWord] = tempTF
end
##build postive dictionary
##open mega positive doc
f = open("PosVocab.txt")
lines = readlines(f)
close(f)
#chomp each line
for i = 1:length(lines)
    lines[i] = chomp(lines[i])
end
#create Post dictionary with tf
PosDictionary = Dict()
for i = 1:length(lines)
    tempArray = split(lines[i])
    tempWord = ""
    tempWord = tempArray[1]
    #tf
    tempTF = parse(tempArray[2])
    PosDictionary[tempWord] = tempTF
end
##build dictionary
##open vocab files
f = open("vocab.txt")
lines = readlines(f)
close(f)
lines
#chomp each line
for i = 1:length(lines)
    lines[i] = chomp(lines[i])
end
#create dictionary with tf
Vocab = Dict()
for i = 1:length(lines)
    tempArray = split(lines[i])
    tempWord = ""
    tempWord = tempArray[1]
    tempTF = parse(tempArray[2])
    Vocab[tempWord] = tempTF
end
###################################################################################################
#read in review
function generateReview(fileName)
    f = open(fileName)
    lines = readlines(f)
    close(f)
    #chomp each line
    review = ""
    for i = 1:length(lines)
        tempString = lines[i]
        tempString = chomp(tempString)
        tempString = replace(tempString,"\"","")
        tempString = replace(tempString,"(","")
        tempString = replace(tempString,")","")
        tempString = replace(tempString,".","")
        tempString = replace(tempString,",","")
        tempString = replace(tempString,":","")
        tempString = replace(tempString,";","")
        tempString = replace(tempString,"  "," ")
        review = review * tempString
    end
    return review
end
###############################################################################################
#dictionary of postive words and negative words
f = open("posnegwords.txt")
lines = readlines(f)
close(f)
PosNegWords = Dict()
for i = 1:length(lines)
    lines[i] = chomp(lines[i])
    lines[i] = replace(lines[i],"\t","  ")
    lines[i] = replace(lines[i],"  "," ")
    tempArray = split(lines[i])
    for w in tempArray
        if(w == "Negativ")
            tempArray[1] = replace(tempArray[1],"#","")
            tempArray[1] = replace(tempArray[1],"1","")
            tempArray[1] = replace(tempArray[1],"2","")
            tempArray[1] = replace(tempArray[1],"3","")
            tempArray[1] = replace(tempArray[1],"4","")
            tempArray[1] = replace(tempArray[1],"5","")
            tempArray[1] = replace(tempArray[1],"6","")
            tempArray[1] = replace(tempArray[1],"7","")
            tempArray[1] = replace(tempArray[1],"8","")
            tempArray[1] = replace(tempArray[1],"9","")
            tempArray[1] = replace(tempArray[1],"10","")
            PosNegWords[lowercase(tempArray[1])] = -1
        end
        if(w == "Positiv")
            tempArray[1] = replace(tempArray[1],"#","")
            tempArray[1] = replace(tempArray[1],"1","")
            tempArray[1] = replace(tempArray[1],"2","")
            tempArray[1] = replace(tempArray[1],"3","")
            tempArray[1] = replace(tempArray[1],"4","")
            tempArray[1] = replace(tempArray[1],"5","")
            tempArray[1] = replace(tempArray[1],"6","")
            tempArray[1] = replace(tempArray[1],"7","")
            tempArray[1] = replace(tempArray[1],"8","")
            tempArray[1] = replace(tempArray[1],"9","")
            tempArray[1] = replace(tempArray[1],"10","")
            PosNegWords[lowercase(tempArray[1])] = 1
        end
    end
end
###################################################################################################
#Learning
#compute parameter
function ComputeParaEst(NegDictionary,PosDictionary,Vocab)
    VocabSize = length(Vocab)
    termGNeg = Dict()
    termGPos = Dict()
    NegSize = 0 
    PosSize = 0 
    for (k,tf) in NegDictionary
        NegSize = NegSize + tf
    end
    
    for (k,tf) in PosDictionary
        PosSize = PosSize + tf
    end
    
    for (k,tf) in Vocab
        nk1 = try NegDictionary[k] catch 0 end #find neg words
        nk2 = try PosDictionary[k] catch 0 end #find pos words
#         NP = try PosNegWords[k] catch 0 end
#         if (NP != 0)
#             if (NP == 1)  nk2 = nk2*100 end #upshoot positve words
#             if (NP == -1) nk1 = nk1*100 end #upshoot negative words
#         end
        pwc1 =(nk1+1)/(NegSize+VocabSize) #prob word given class 1 
        pwc2 =(nk2+1)/(PosSize+VocabSize) #prob word given class 2
        termGNeg[k] = pwc1
        termGPos[k] = pwc2
    end
    return termGNeg,termGPos
end
termGNeg,termGPos = ComputeParaEst(NegDictionary,PosDictionary,Vocab);

In [203]:
#MULTINOMIAL NAIVE BAYES
function NaiveBayes(review,termGNeg,termGPos,NegDictionary,
                    PosDictionary,Vocab,PosNegWords)
    #VocabSize
    VocabSize = length(termGNeg)
    #break down review into word.
    listOfWord = split(review)
    #prior 
    pN = 1/2
    pP = 1/2
    scoreN = log10(pN)
    scoreP = log10(pP)
    #spagetti code, compute negvocab size
    NegSize = 0 
    PosSize = 0 
    for (k,tf) in NegDictionary
        NegSize = NegSize + tf
    end
    
    for (k,tf) in PosDictionary
        PosSize = PosSize + tf
    end
    #compute
    for w in listOfWord
        wN = try termGNeg[w] catch 1/(NegSize+VocabSize) end #dealing with unknown words
        wP = try termGPos[w] catch 1/(PosSize+VocabSize) end#dealing with unknown words
        #upweight
        NP = try PosNegWords[w] catch 0 end
        if (NP != 0)
            if (NP == 1)  wP = wP*2 end #upshoot positve words
            if (NP == -1) wN = wN*2 end #upshoot negative words
        end
        #end upweight
        scoreN = scoreN + log10(wN) 
        scoreP = scoreP + log10(wP) 
    end
    if (scoreN > scoreP) 
        return "-"
    else    
        return "+"
    end
end

NaiveBayes (generic function with 1 method)

In [204]:
NegCount = 0
PosCount = 0
half = 3200 ###choosing how many reviews to evaluate
size = half*2
for i = 1:half
    fileName ="test_set1\\pos\\pos ($i).txt"
    try
        review = generateReview(fileName)
        r = NaiveBayes(review,termGNeg,termGPos,NegDictionary,PosDictionary,Vocab,PosNegWords)
        if (r == "+")
            PosCount +=1
        else
            NegCount +=1
        end
    catch
        println(i)
    end
end
NegCount1 = 0
PosCount1 = 0
for i = 1:half
    fileName ="test_set1\\neg\\neg ($i).txt"
    try
        review = generateReview(fileName)
        r = NaiveBayes(review,termGNeg,termGPos,NegDictionary,PosDictionary,Vocab,PosNegWords)
        if (r == "+")
            PosCount1 +=1
        else
            NegCount1 +=1
        end
    catch
        println(i)
    end
end

In [205]:
(PosCount + NegCount1)/size #compute accuracy

0.70515625

In [195]:
println(PosCount,"\t",size-PosCount)

1459	10541


In [184]:
#sample outpur
for i = 1:20
    fileName ="test_set1\\pos\\pos ($i).txt"
    review = generateReview(fileName)
    println(i,"\t\t",NaiveBayes(review,termGNeg,termGPos,NegDictionary,PosDictionary,Vocab,PosNegWords))
end

1		+
2		-
3		+
4		-
5		-
6		-
7		-
8		+
9		-
10		-
11		+
12		+
13		+
14		+
15		+
16		-
17		-
18		+
19		+
20		+


In [ ]:
#####################################
#Binary Multinomial after this point 


In [115]:
################################################################################
#Binarized Naive Bayes
##Preprocess the Data
##Generate negative review
##open mega negative doc
f = open("NegVocab.txt")
lines = readlines(f)
close(f)
#chomp each line
for i = 1:length(lines)
    lines[i] = chomp(lines[i])
end
#create negative dictionary with tf
NegDictionary = Dict()
for i = 1:length(lines)
    tempArray = split(lines[i])
    tempWord = ""
    tempWord = tempArray[1]
    #df
    tempDF = parse(tempArray[3])
    NegDictionary[tempWord] = tempDF
end
##build postive dictionary
##open mega positive doc
f = open("PosVocab.txt")
lines = readlines(f)
close(f)
#chomp each line
for i = 1:length(lines)
    lines[i] = chomp(lines[i])
end
#create Post dictionary with tf
PosDictionary = Dict()
for i = 1:length(lines)
    tempArray = split(lines[i])
    tempWord = ""
    tempWord = tempArray[1]
    #df
    tempDF = parse(tempArray[3])
    PosDictionary[tempWord] = tempDF
end
##build dictionary
##open vocab files
f = open("vocab.txt")
lines = readlines(f)
close(f)
lines
#chomp each line
for i = 1:length(lines)
    lines[i] = chomp(lines[i])
end
#create dictionary with tf
Vocab = Dict()
for i = 1:length(lines)
    tempArray = split(lines[i])
    tempWord = ""
    tempWord = tempArray[1]
    #df
    tempDF = parse(tempArray[3])
    Vocab[tempWord] = tempDF
end
###################################################################################################
#Learning
#compute parameter
function ComputeParaEst(NegDictionary,PosDictionary,Vocab)
    VocabSize = length(Vocab)
    termGNeg = Dict()
    termGPos = Dict()
    NegSize = 0 
    PosSize = 0 
    for (k,tf) in NegDictionary
        NegSize = NegSize + tf
    end
    
    for (k,tf) in PosDictionary
        PosSize = PosSize + tf
    end
    
    for (k,tf) in Vocab
        nk1 = try NegDictionary[k] catch 0 end #find neg words
        nk2 = try PosDictionary[k] catch 0 end #find pos words
        pwc1 =(nk1+1)/(NegSize+VocabSize) #prob word given class 1 
        pwc2 =(nk2+1)/(PosSize+VocabSize) #prob word given class 2
        termGNeg[k] = pwc1
        termGPos[k] = pwc2
    end
    return termGNeg,termGPos
end
termGNeg,termGPos = ComputeParaEst(NegDictionary,PosDictionary,Vocab);

In [185]:
#Binary multinomial Naive Bayes
function BinarizedNaiveBayes(review,termGNeg,termGPos,NegDictionary,
                             PosDictionary,Vocab,PosNegWords)
    #VocabSize
    VocabSize = length(termGNeg)
    #break down review into word.
    listOfWord = split(review)
    reviewDict = Dict()
    #for each word review
    for w in  listOfWord
        reviewDict[w] = 1
    end
    #prior 
    pN = 1/2
    pP = 1/2
    scoreN = log10(pN)
    scoreP = log10(pP)
    #spagetti code, compute negvocab size
    NegSize = 0 
    PosSize = 0 
    for (k,tf) in NegDictionary
        NegSize = NegSize + tf
    end
    
    for (k,tf) in PosDictionary
        PosSize = PosSize + tf
    end
    
    #compute
    for (w,v) in reviewDict
        wN = try termGNeg[w] catch 1/(NegSize+VocabSize) end #dealing with unknown words
        wP = try termGPos[w] catch 1/(PosSize+VocabSize) end #dealing with unknown words
        #upweight
#         NP = try PosNegWords[w] catch 0 end
#         if (NP != 0)
#             if (NP == 1)  wP = wP*3 end #upshoot positve words
#             if (NP == -1) wN = wN*3 end #upshoot negative words
#         end
        #upweight
        scoreN = scoreN + log10(wN) 
        scoreP = scoreP + log10(wP) 
    end
    if (scoreN > scoreP) 
        return "-"
    else    
        return "+"
    end
end

BinarizedNaiveBayes (generic function with 1 method)

In [188]:
NegCount = 0
PosCount = 0
half = 3200
size = half*2
for i = 1:half
    fileName ="test_set1\\pos\\pos ($i).txt"
    try
        review = generateReview(fileName)
        r = BinarizedNaiveBayes(review,termGNeg,termGPos,NegDictionary,PosDictionary,Vocab,PosNegWords)
        if (r == "+")
            PosCount +=1
        else
            NegCount +=1
        end
    catch
        println(i)
    end
end

NegCount1 = 0
PosCount1 = 0
for i = 1:half
    fileName ="test_set1\\neg\\neg ($i).txt"
    try
        review = generateReview(fileName)
        r = BinarizedNaiveBayes(review,termGNeg,termGPos,NegDictionary,PosDictionary,Vocab,PosNegWords)
        if (r == "+")
            PosCount1 +=1
        else
            NegCount1 +=1
        end
    catch
        println(i)
    end
end

In [189]:
(PosCount + NegCount1)/size

0.70875

In [190]:
println(PosCount,"\t",size-PosCount)

1465	4935


In [179]:
x = [200;800;1600;3200;6400;12000];
y1 = [.695;.676;.642;.622;.614;616];
y2 = [.765;.771;.733;.714;.703;.711];
y3 = [.790;.760;.710;.700;.708;.709];
y4 = [.840;.830;.799;.789;.794;.793];

1x4 Array{Float64,2}:
 0.84  0.83  0.799  0.789

In [182]:
using Plots
plot(x,y1,marker=([:hex :d]),lab="NB")
plot!(x,y2,marker=([:hex :d]),lab="BNB")
plot!(x,y3,marker=([:hex :d]),lab="UNB")
plot!(x,y4,marker=([:hex :d]),lab="CNB")
savefig("Accuracy.pdf")

<?xml version="1.0" encoding="utf-8" standalone="no"?>
<!DOCTYPE svg PUBLIC "-//W3C//DTD SVG 1.1//EN"
 "http://www.w3.org/Graphics/SVG/1.1/DTD/svg11.dtd">
<!-- Created with matplotlib (http://matplotlib.org/) -->
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 <!-- 0.62 -->
 
 
 <path d="M 33.015625 40.375 
Q 26.375 40.375 22.484375 35.828125 
Q 18.609375 31.296875 18.609375 23.390625 
Q 18.609375 15.53125 22.484375 10.953125 
Q 26.375 6.390625 33.015625 6.390625 
Q 39.65625 6.390625 43.53125 10.953125 
Q 47.40625 15.53125 47.40625 23.390625 
Q 47.40625 31.296875 43.53125 35.828125 
Q 39.65625 40.375 33.015625 40.375 
M 52.59375 71.296875 
L 52.59375 62.3125 
Q 48.875 64.0625 45.09375 64.984375 
Q 41.3125 65.921875 37.59375 65.921875 
Q 27.828125 65.921875 22.671875 59.328125 
Q 17.53125 52.734375 16.796875 39.40625 
Q 19.671875 43.65625 24.015625 45.921875 
Q 28.375 48.1875 33.59375 48.1875 
Q 44.578125 48.1875 50.953125 41.515625 
Q 57.328125 34.859375 57.328125 23.390625 
Q 57.328125 12.15625 50.6875 5.359375 
Q 44.046875 -1.421875 33.015625 -1.421875 
Q 20.359375 -1.421875 13.671875 8.265625 
Q 6.984375 17.96875 6.984375 36.375 
Q 6.984375 53.65625 15.1875 63.9375 
Q 23.390625 74.21875 37.203125 74.21875 
Q 40.921875 74.21875 44.703125 73.484375 
Q 48.484375 72.75 52.59375 71.296875 
" id="BitstreamVeraSans-Roman-36"/>
 <path d="M 31.78125 66.40625 
Q 24.171875 66.40625 20.328125 58.90625 
Q 16.5 51.421875 16.5 36.375 
Q 16.5 21.390625 20.328125 13.890625 
Q 24.171875 6.390625 31.78125 6.390625 
Q 39.453125 6.390625 43.28125 13.890625 
Q 47.125 21.390625 47.125 36.375 
Q 47.125 51.421875 43.28125 58.90625 
Q 39.453125 66.40625 31.78125 66.40625 
M 31.78125 74.21875 
Q 44.046875 74.21875 50.515625 64.515625 
Q 56.984375 54.828125 56.984375 36.375 
Q 56.984375 17.96875 50.515625 8.265625 
Q 44.046875 -1.421875 31.78125 -1.421875 
Q 19.53125 -1.421875 13.0625 8.265625 
Q 6.59375 17.96875 6.59375 36.375 
Q 6.59375 54.828125 13.0625 64.515625 
Q 19.53125 74.21875 31.78125 74.21875 
" id="BitstreamVeraSans-Roman-30"/>
 <path d="M 19.1875 8.296875 
L 53.609375 8.296875 
L 53.609375 0 
L 7.328125 0 
L 7.328125 8.296875 
Q 12.9375 14.109375 22.625 23.890625 
Q 32.328125 33.6875 34.8125 36.53125 
Q 39.546875 41.84375 41.421875 45.53125 
Q 43.3125 49.21875 43.3125 52.78125 
Q 43.3125 58.59375 39.234375 62.25 
Q 35.15625 65.921875 28.609375 65.921875 
Q 23.96875 65.921875 18.8125 64.3125 
Q 13.671875 62.703125 7.8125 59.421875 
L 7.8125 69.390625 
Q 13.765625 71.78125 18.9375 73 
Q 24.125 74.21875 28.421875 74.21875 
Q 39.75 74.21875 46.484375 68.546875 
Q 53.21875 62.890625 53.21875 53.421875 
Q 53.21875 48.921875 51.53125 44.890625 
Q 49.859375 40.875 45.40625 35.40625 
Q 44.1875 33.984375 37.640625 27.21875 
Q 31.109375 20.453125 19.1875 8.296875 
" id="BitstreamVeraSans-Roman-32"/>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 <!-- 0.63 -->
 
 <path d="M 40.578125 39.3125 
Q 47.65625 37.796875 51.625 33 
Q 55.609375 28.21875 55.609375 21.1875 
Q 55.609375 10.40625 48.1875 4.484375 
Q 40.765625 -1.421875 27.09375 -1.421875 
Q 22.515625 -1.421875 17.65625 -0.515625 
Q 12.796875 0.390625 7.625 2.203125 
L 7.625 11.71875 
Q 11.71875 9.328125 16.59375 8.109375 
Q 21.484375 6.890625 26.8125 6.890625 
Q 36.078125 6.890625 40.9375 10.546875 
Q 45.796875 14.203125 45.796875 21.1875 
Q 45.796875 27.640625 41.28125 31.265625 
Q 36.765625 34.90625 28.71875 34.90625 
L 20.21875 34.90625 
L 20.21875 43.015625 
L 29.109375 43.015625 
Q 36.375 43.015625 40.234375 45.921875 
Q 44.09375 48.828125 44.09375 54.296875 
Q 44.09375 59.90625 40.109375 62.90625 
Q 36.140625 65.921875 28.71875 65.921875 
Q 24.65625 65.921875 20.015625 65.03125 
Q 15.375 64.15625 9.8125 62.3125 
L 9.8125 71.09375 
Q 15.4375 72.65625 20.34375 73.4375 
Q 25.25 74.21875 29.59375 74.21875 
Q 40